In [1]:
import networkx as nx
twitter: nx.DiGraph = nx.read_edgelist("./ego-Twitter/twitter_combined.txt.gz", create_using=nx.DiGraph)
twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

(81306, 1768149, True)

In [2]:
#サンプリングしたグラフからコミュニティ検出（louvain法）
import networkx as nx

twitter: nx.DiGraph = nx.read_edgelist("sampled_twitter_graph.txt", create_using=nx.DiGraph)

twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

twitter_comms = nx.community.louvain_communities(twitter)

print(f"検出されたコミュニティ数: {len(twitter_comms)}")

検出されたコミュニティ数: 51


In [3]:
import os
import json # JSONライブラリを使用
import numpy as np
# (networkx as nx もインポートされている前提)

# --- 1. 保存用フォルダの作成 ---
output_folder = "adj_json_communities_txt" # 保存先のフォルダ名
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"\n保存用フォルダ '{output_folder}' を作成しました。")
else:
    print(f"\n保存用フォルダ '{output_folder}' は既に存在します。")


# --- 2. 各コミュニティを指定の形式で .txt ファイルに保存 ---
print("各コミュニティを隣接行列の辞書形式で、txtファイルに保存しています...")

# twitter_comms が定義されている前提でループを開始
for i, community_nodes in enumerate(twitter_comms):
    if len(community_nodes) < 2:
        continue
    
    # Step 2-1: サブグラフを生成
    subgraph = twitter.subgraph(community_nodes)
    
    # Step 2-2: サブグラフから隣接行列を生成
    node_list = list(subgraph.nodes())
    adj_matrix_np = nx.to_numpy_array(subgraph, nodelist=node_list, dtype=int)
    
    # Step 2-3: NumPy配列をご希望の辞書形式に変換
    adjacency_dict = {}
    for row_idx, source_node in enumerate(node_list):
        row_dict = {}
        for col_idx, target_node in enumerate(node_list):
            row_dict[target_node] = int(adj_matrix_np[row_idx, col_idx])
        
        adjacency_dict[source_node] = row_dict
        
    # Step 2-4: ファイルパスを定義 (★拡張子を .txt に)
    file_path = os.path.join(output_folder, f"community_{i}.txt")
    
    # Step 2-5: TXTファイルとして書き出す (★json.dumpを使用)
    with open(file_path, 'w', encoding='utf-8') as f:
        # json.dumpで書き出すと、ご提示の例の形式と一致します
        # indent=2 を指定すると、例のように整形されて見やすくなります
        json.dump(adjacency_dict, f, indent=2) 

print(f"\n処理完了: {len(twitter_comms)}個のコミュニティを '{output_folder}' フォルダにTXTファイルとして保存しました。")


保存用フォルダ 'adj_json_communities_txt' を作成しました。
各コミュニティを隣接行列の辞書形式で、txtファイルに保存しています...

処理完了: 51個のコミュニティを 'adj_json_communities_txt' フォルダにTXTファイルとして保存しました。


In [4]:
import os
import json

# TXTファイルが保存されているフォルダ名
folder_name = "adj_json_communities_txt" 

# フォルダの存在確認
if not os.path.exists(folder_name):
    print(f"エラー: フォルダ '{folder_name}' が見つかりません。")
    print("先にファイル保存コードを実行してください。")
    exit()

print(f"フォルダ '{folder_name}' 内のファイルを確認します...\n")

try:
    txt_files = [f for f in os.listdir(folder_name) if f.endswith('.txt')]
    if not txt_files:
        print("エラー: フォルダ内にTXTファイルが見つかりません。")
        exit()
except Exception as e:
    print(f"エラー: フォルダの読み込み中に問題が発生しました。 {e}")
    exit()

# すべてのファイルが有向グラフであったかを追跡するフラグ
all_are_directed = True

# --- 各TXTファイルをループして検証 ---
for filename in txt_files:
    file_path = os.path.join(folder_name, filename)
    
    print(f"--- 検証中: {filename} ---")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            # JSON形式のテキストとして読み込む
            adj_dict = json.load(f)

        # ノード数が2未満の場合は比較できないためスキップ
        if len(adj_dict) < 2:
            print("  => ⚠️ スキップ (ノード数が2未満のため比較不可)。")
            print("-" * 35)
            continue

        # ノードのリストを取得
        nodes = list(adj_dict.keys())
        is_symmetric = True # まずは「対称である(無向)」と仮定
        asymmetry_proof = None # 非対称である証拠

        # --- 行列が対称(Symmetric)か非対称(Asymmetric)かチェック ---
        # (A, B)のすべてのペアをチェック
        for i in range(len(nodes)):
            for j in range(i + 1, len(nodes)): # i+1から始めることで重複チェックを避ける
                node_a = nodes[i]
                node_b = nodes[j]
                
                # A->B の値と B->A の値が異なるか？
                if adj_dict[node_a][node_b] != adj_dict[node_b][node_a]:
                    # 異なっていたら、非対称(有向グラフ)である
                    is_symmetric = False
                    asymmetry_proof = (node_a, node_b) # 証拠を保存
                    break # このファイルのチェックはここで十分
            
            if not is_symmetric:
                break # 外側のループも抜ける

        # --- 検証結果の出力 ---
        if not is_symmetric:
            # 非対称 = 有向グラフ
            print(f"  ✅ 結果: これは有向グラフです。")
            print(f"     理由: 隣接行列が非対称(Asymmetric)です。")
            proof_a, proof_b = asymmetry_proof
            print(f"     証拠: ノード {proof_a} -> {proof_b} は [{adj_dict[proof_a][proof_b]}] ですが、")
            print(f"           ノード {proof_b} -> {proof_a} は [{adj_dict[proof_b][proof_a]}] です。")
        else:
            # 対称 = 無向グラフ
            print(f"  ❌ 結果: これは有向グラフではありません (実質的に無向グラフです)。")
            print(f"     理由: 隣接行列が対称(Symmetric)です。")
            print(f"     詳細: すべてのノードペア(A, B)において、 A -> B への接続値と")
            print(f"           B -> A への接続値が完全に一致しています (全て相互関係です)。")
            all_are_directed = False

    except Exception as e:
        print(f"  => ❌ エラー: ファイル処理中に問題が発生しました。")
        print(f"     エラー内容: {e}")
        all_are_directed = False
    
    print("-" * 35)

# --- 最終結論 ---
print("\n--- 総合結果 ---")
if all_are_directed:
    print("✅ 分析したすべてのコミュニティが、有向グラフの性質（非対称な関係）を保持していました。")
else:
    print("⚠️ 少なくとも1つのコミュニティが、完全に対称的（無向グラフ）であることが判明しました。")

フォルダ 'adj_json_communities_txt' 内のファイルを確認します...

--- 検証中: community_1.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 14780587 -> 24815332 は [0] ですが、
           ノード 24815332 -> 14780587 は [1] です。
-----------------------------------
--- 検証中: community_0.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 2710231 -> 14060856 は [0] ですが、
           ノード 14060856 -> 2710231 は [1] です。
-----------------------------------
--- 検証中: community_2.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 23016199 -> 86572842 は [0] ですが、
           ノード 86572842 -> 23016199 は [1] です。
-----------------------------------
--- 検証中: community_3.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 236657680 -> 29877307 は [1] ですが、
           ノード 29877307 -> 236657680 は [0] です。
-----------------------------------
--- 検証中: community_7.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 19167293 -> 18812572